<a href="https://colab.research.google.com/github/ben854719/Autonomous-Health-Monitoring-Emergency-Response/blob/main/Agentic_AI_Emergency_Protocols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install --upgrade langchain-google-genai google-generativeai
!pip install --upgrade langchain-google-genai google-generativeai langgraph

  Using cached langchain_google_genai-2.1.12-py3-none-any.whl.metadata (7.1 kB)
  Using cached google_ai_generativelanguage-0.7.0-py3-none-any.whl.metadata (10 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.0-py3-none-any.whl.metadata (3.9 kB)
INFO: This is taking longer than usual. You might need to prov

In [6]:
!pip install "mcp[cli]"
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("GeminiTools")

@mcp.tool()
def search(query: str) -> list:
    # Your search logic here
    return ["Result 1", "Result 2"]

In [7]:
!cd mcp-server-demo
!ls

/bin/bash: line 1: cd: mcp-server-demo: No such file or directory
sample_data


In [8]:
!cd mcp-server-demo && uv add langchain-google-genai langgraph

/bin/bash: line 1: cd: mcp-server-demo: No such file or directory


In [9]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 5.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=b9a7eb23c8c50f64f6cd6ac0702e54979f475e7f3b37da85858ba882bc618bae
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1

In [22]:
from IPython import get_ipython
from IPython import display
import os
import requests
import nltk
from urllib.parse import quote
from langgraph.graph import StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from unstructured.partition.html import partition_html
from google.colab import userdata
from mcp.server.fastmcp import FastMCP

# Download necessary NLTK data
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

# Load Gemini API key
api_key = userdata.get("Ben856")
if not api_key:
    raise ValueError("Ben856 secret not found. Please set your API key in Colab Secrets.")

# Initialize Gemini model
gemini_model = ChatGoogleGenerativeAI(model='gemini-2.5-flash', api_key=api_key)

# Optional: Initialize FastMCP (placeholder for future use)
# mcp_instance = FastMCP(config_path="path/to/config.yaml")

# Node 1: Fetch and process web content
def fetch_and_process_web_content(state):
    url = state.get("url")
    if not url:
        return {"url": "", "text": "Error: URL missing from state."}

    try:
        safe_url = quote(url, safe=":/?=&")
        response = requests.get(safe_url, timeout=20)
        response.raise_for_status()

        elements = partition_html(text=response.text)
        text_content = "\n".join([str(el) for el in elements])

        return {"url": url, "text": text_content}

    except requests.exceptions.MissingSchema:
        return {"url": url, "text": "Error: Invalid URL format. Use http:// or https://"}
    except requests.exceptions.ConnectionError:
        return {"url": url, "text": f"Error: Could not connect to {url}"}
    except requests.exceptions.Timeout:
        return {"url": url, "text": f"Error: Request to {url} timed out."}
    except requests.exceptions.RequestException as e:
        return {"url": url, "text": f"Error: Request error: {e}"}
    except Exception as e:
        return {"url": url, "text": f"Unexpected error: {e}"}

# Node 2: Analyze content using Gemini
def analyze_content(state):
    url = state.get("url", "No URL provided")
    text = state.get("text", "No text content found.")

    prompt_text = f"""
    Analyze the following text content from the URL: {url}

    Content:
    {text}

    Provide a summary of the content and identify the main topics discussed.
    """

    try:
        response = gemini_model.invoke([HumanMessage(content=prompt_text)])
        analysis_result = response.content
    except KeyboardInterrupt:
        analysis_result = f"Analysis interrupted for URL: {url}"
    except Exception as e:
        analysis_result = f"Error during Gemini analysis: {e}"

    return {"url": url, "text": text, "analysis": analysis_result}

# Define LangGraph workflow
workflow = StateGraph(dict)
workflow.add_node("fetch_web_content", fetch_and_process_web_content)
workflow.add_node("summarize_content", analyze_content)
workflow.set_entry_point("fetch_web_content")
workflow.add_edge("fetch_web_content", "summarize_content")
executable_graph = workflow.compile()

# List of URLs to process
urls = [
    "https://www.ccohs.ca/oshanswers/legisl/legislation/injury_reporting.html",
    "https://news.ontario.ca/en/release/1006370/ontario-beginning-construction-of-highway-413",
    "https://www.opp.ca/index.php?id=125",
]

# Run workflow for each URL
for url in urls:
    print(f"\n Processing: {url}")
    final_state = executable_graph.invoke({"url": url})
    print(f"\n Summary:\n{final_state['analysis']}")


 Processing: https://www.ccohs.ca/oshanswers/legisl/legislation/injury_reporting.html

 Summary:
The provided CCOHS document, "Health and Safety Legislation in Canada - Injury Reporting," offers a comprehensive guide for employers regarding their responsibilities when workplace injuries or serious incidents occur. It details the reporting requirements, immediate actions to take, support for workers, classification of injuries, and return-to-work obligations.

### Summary of the Content:

The document outlines the **reporting requirements** for workplace injuries and illnesses in Canada. Employers must report to their jurisdiction's workers' compensation board if an injury or illness requires medical attention, leads to missed work shifts, or necessitates modified duties. For **serious incidents, injuries, and fatalities**, additional reporting to the government department responsible for health and safety is mandatory, triggering investigation requirements. The text emphasizes verifyi